In [39]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [40]:
tf.set_random_seed(333)

In [41]:
sentence = ('''이제 좀 구체적인 이야기를 해보죠. scikit-learn 0.18 버전에 다층 퍼셉트론(Multi-layer Perceptron)이 추가되었습니다.
향후 CNN, RNN 모델이 추가되거나 GPU지원이 될까요? 아니요. 이미 파이썬에서 사용할 수 있는 훌륭한 딥러닝 라이브러리들이 있고
이 분야는 매우 빠르게 변합니다. 우리가 케라스나 텐서플로를 따라잡아야 할 이유가 없습니다. GPU를 사용하는 데 필요한 패키지를 
설치하는 일이 여전히 큰 장벽이라서 GPU 지원은 추가하지 않기로 했습니다. 또 GPU를 지원한다는 것은 근본적으로 알고리즘들을 
밑바닥부터 다시 작성해야 한다는 뜻입니다. 다른 패키지를 사용해 쉽게 구현할 수 있는데, scikit-learn에서 굳이 이렇게 해야 할 강한 동기를
찾지 못했습니다.''')
#sklearn은 안됨!

In [44]:
char_set = list(set(sentence))
char_set[:10] #이걸 데이터 하나로 치고 이 다음에는 이부터 시작한다!

['데', '나', '이', 'l', 'N', '모', '작', '층', 'a', '치']

In [43]:
len(sentence), len(char_set)

(400, 153)

In [6]:
char_dic = {w: i for i, w in enumerate(char_set)}
char_dic

{'데': 0,
 '나': 1,
 '이': 2,
 'l': 3,
 'N': 4,
 '모': 5,
 '작': 6,
 '층': 7,
 'a': 8,
 '치': 9,
 '시': 10,
 '구': 11,
 '리': 12,
 '수': 13,
 '8': 14,
 '입': 15,
 '기': 16,
 '사': 17,
 '썬': 18,
 '뜻': 19,
 '유': 20,
 'G': 21,
 '용': 22,
 '필': 23,
 '본': 24,
 '라': 25,
 '합': 26,
 '지': 27,
 'p': 28,
 '빠': 29,
 '추': 30,
 '에': 31,
 '있': 32,
 '딥': 33,
 ',': 34,
 '밑': 35,
 '것': 36,
 '현': 37,
 '후': 38,
 '원': 39,
 '거': 40,
 '될': 41,
 '설': 42,
 '.': 43,
 '굳': 44,
 '우': 45,
 '장': 46,
 '들': 47,
 '변': 48,
 '잡': 49,
 '전': 50,
 '\n': 51,
 '로': 52,
 '미': 53,
 '할': 54,
 '하': 55,
 '키': 56,
 '야': 57,
 '벽': 58,
 '죠': 59,
 '한': 60,
 'o': 61,
 '은': 62,
 '론': 63,
 '쉽': 64,
 '요': 65,
 '게': 66,
 '(': 67,
 '따': 68,
 'y': 69,
 '강': 70,
 '닝': 71,
 '니': 72,
 '했': 73,
 '여': 74,
 '즘': 75,
 '제': 76,
 '다': 77,
 '륭': 78,
 '른': 79,
 '좀': 80,
 '1': 81,
 '찾': 82,
 '브': 83,
 '되': 84,
 '부': 85,
 '적': 86,
 'r': 87,
 '보': 88,
 '알': 89,
 '0': 90,
 '트': 91,
 '터': 92,
 '러': 93,
 'u': 94,
 'C': 95,
 '케': 96,
 '훌': 97,
 'n': 98,
 '가': 99,
 '퍼': 100

In [7]:
data_dim = len(char_set) # 데이터 개수
hidden_size = len(char_set) 
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [8]:
X_data = []
Y_data = []

In [26]:
sentence[0:10], sentence[1:11]

('이제 좀 구체적인 ', '제 좀 구체적인 이')

In [27]:
for c in sentence[0:10]:
    print(c, end=',')
    print(char_dic[c])

이,2
제,76
 ,108
좀,80
 ,108
구,11
체,144
적,86
인,116
 ,108


In [9]:
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1] #X값에 1씩 더한 값
    
    if i%100 == 0:
        print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index //0부터 9까지 가는데 문자 C를 줘서 인덱싱을 가져가겠다는 말
    y = [char_dic[c] for c in y_str]  # y str to index

    X_data.append(x) #index list를 추가
    Y_data.append(y) #얘도 같음

0 이제 좀 구체적인  -> 제 좀 구체적인 이
100 추가되거나 GPU지 -> 가되거나 GPU지원
200 할 이유가 없습니다 ->  이유가 없습니다.
300 고리즘들을 
밑바닥 -> 리즘들을 
밑바닥부


In [14]:
batch_size = len(X_data) #데이타가 별로 안되니까 전체 set을 배치로 담음

In [16]:
X_data #얘처럼 데이터를 변환해줌.

[[2, 76, 108, 80, 108, 11, 144, 86, 116, 108],
 [76, 108, 80, 108, 11, 144, 86, 116, 108, 2],
 [108, 80, 108, 11, 144, 86, 116, 108, 2, 57],
 [80, 108, 11, 144, 86, 116, 108, 2, 57, 16],
 [108, 11, 144, 86, 116, 108, 2, 57, 16, 145],
 [11, 144, 86, 116, 108, 2, 57, 16, 145, 108],
 [144, 86, 116, 108, 2, 57, 16, 145, 108, 117],
 [86, 116, 108, 2, 57, 16, 145, 108, 117, 88],
 [116, 108, 2, 57, 16, 145, 108, 117, 88, 59],
 [108, 2, 57, 16, 145, 108, 117, 88, 59, 43],
 [2, 57, 16, 145, 108, 117, 88, 59, 43, 108],
 [57, 16, 145, 108, 117, 88, 59, 43, 108, 106],
 [16, 145, 108, 117, 88, 59, 43, 108, 106, 124],
 [145, 108, 117, 88, 59, 43, 108, 106, 124, 107],
 [108, 117, 88, 59, 43, 108, 106, 124, 107, 146],
 [117, 88, 59, 43, 108, 106, 124, 107, 146, 107],
 [88, 59, 43, 108, 106, 124, 107, 146, 107, 125],
 [59, 43, 108, 106, 124, 107, 146, 107, 125, 148],
 [43, 108, 106, 124, 107, 146, 107, 125, 148, 3],
 [108, 106, 124, 107, 146, 107, 125, 148, 3, 149],
 [106, 124, 107, 146, 107, 125, 148,

In [28]:
batch_size = len(X_data)
batch_size

390

In [12]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

In [25]:
X_one_hot = tf.one_hot(X, num_classes) #one-hot인코딩을 이렇게 한방에 가능. 우리가 일일히 해 줄 필요가 없다.
with tf.Session() as sess_print:
    X_one_hot = sess_print.run(X_one_hot[0, 0], feed_dict= {X: X_data})
    print(X_one_hot)
    print(len(X_one_hot))

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
153


In [17]:
#cell을 lstm으로 설정, 또 여럿 있었으면 좋겠다->multiRNN, 3층으로
lstm_cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True) #lstm쓸거야
multi_cells = rnn.MultiRNNCell([lstm_cell] *3, state_is_tuple=True) #3층으로

W0628 15:58:57.308463  1296 deprecation.py:323] From <ipython-input-17-7f93a31bca0b>:1: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0628 15:58:57.308463  1296 deprecation.py:323] From <ipython-input-17-7f93a31bca0b>:2: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0628 15:58:57.308463  1296 rnn_cell_impl.py:1642] At least two cells provided to MultiRNNCell are the same object and will share weights.


In [18]:
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32) #dynamic으로

W0628 15:59:02.212665  1296 deprecation.py:323] From <ipython-input-18-c72e086eae2f>:1: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0628 15:59:02.562474  1296 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0628 15:59:02.578074  1296 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\python\ops\rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call 

In [29]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [30]:
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [31]:
weights = tf.ones([batch_size, sequence_length])

In [32]:
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [34]:
epoches=500

In [35]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(  #hypo가 result 안에 들어간다
        [train_op, mean_loss, outputs], feed_dict={X: X_data, Y: Y_data})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1) #예측값의 최대가 되는 애가 뭐야? 뽑아
#         print(i, j, ''.join([char_set[t] for t in index]), l)

In [36]:
results = sess.run(outputs, feed_dict={X: X_data})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='') #해당 캐릭터셋에 인덱스(=value값)가 지칭하고 있는 단어가뭐야
    else:
        print(char_set[index[-1]], end='')

  좀 구체적인 이야기를 해보죠. scikit-learn에0.18 버전에 다층 퍼셉트론(Multi-layer Perceptron)이 추가되었습니다.
향후 CNN, RNN 모델이 추가되거나 GPU지원이 될까요? 아니요. 이미 파이썬에서 사용할 수 있는 훌륭한 딥러닝 라이브러리들이 있고
이 분야는 매우 빠르게 변합니다. 우리가 케라스나 텐서플로를 따라잡아야 할 이유가 없습니다. GPU를 사용하는 데 필요한 패키지를 
설치하는 일이 여전히 큰 장벽이라서 GPU 지원은 추가하지 않기로 했습니다. 또 GPU를 지원한다는 것은 근본적으로 알고리즘들을 
밑바닥부터 다시 작성해야 한다는 뜻입니다. 다른 패키지를 사용해 쉽게 구현할 수 있는데, scikit-learn에서 굳이 이렇게 해야 할 강한 동기를
찾지 못했습니다.